In [1]:
import pandas as pd
import re
from nltk.corpus import stopwords
import pickle
import numpy as np
from string import punctuation
import time
import spacy
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

## 1. Import all articles containing "regulat*"

In [2]:
# Import all news articles that contain terms "regulat*"
df1=pd.read_pickle('/home/ec2-user/SageMaker/New Uncertainty/parsed_xml.pkl')
print(df1.info())
print(df1.sort_values('StartDate')[['StartDate']])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 822737 entries, 0 to 822736
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   ID             822737 non-null  object        
 1   Title          822737 non-null  object        
 2   Type           822737 non-null  object        
 3   StartDate      822737 non-null  datetime64[ns]
 4   EndDate        822737 non-null  object        
 5   Text           822737 non-null  object        
 6   TextWordCount  822737 non-null  object        
 7   PubTitle       822737 non-null  object        
 8   SourceType     822737 non-null  object        
 9   Year           822737 non-null  int64         
 10  Month          822737 non-null  int64         
 11  Newspaper      822737 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(9)
memory usage: 75.3+ MB
None
        StartDate
0      1985-01-01
78924  1985-01-01
78925  1985-01-01
78926  1985-01

In [3]:
# df2=pd.read_pickle('/home/ec2-user/SageMaker/New Uncertainty/Update Data/parsed_xml_update.pkl')
# print(df2.info())
# print(df2.sort_values('StartDate')[['StartDate']])

In [4]:
# df_all=df1.append(df2).sort_values(['StartDate','Newspaper']).reset_index(drop=True)
# print(df_all.info())

In [5]:
# Remove duplicated articles
IDs_nodup=pd.read_csv('/home/ec2-user/SageMaker/New Uncertainty/IDs_no_duplicates.csv')
IDs_nodup['ID']=IDs_nodup['ID'].astype('str')
df=IDs_nodup.merge(df1,on='ID',how='left')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 788516 entries, 0 to 788515
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   ID             788516 non-null  object        
 1   Title          788516 non-null  object        
 2   Type           788516 non-null  object        
 3   StartDate      788516 non-null  datetime64[ns]
 4   EndDate        788516 non-null  object        
 5   Text           788516 non-null  object        
 6   TextWordCount  788516 non-null  object        
 7   PubTitle       788516 non-null  object        
 8   SourceType     788516 non-null  object        
 9   Year           788516 non-null  int64         
 10  Month          788516 non-null  int64         
 11  Newspaper      788516 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(9)
memory usage: 78.2+ MB
None


## 2. Match BBD words

In [6]:
# BBD's three sets of keywords
uncertain=['uncertainty','uncertain']
econ=['economic','economy']
policy=['regulation', 'deficit','legislation', 'congress', 'white house', 'Federal Reserve', 'the Fed', 'regulations',
        'regulatory', 'deficits','congressional', 'legislative', 'legislature']
regulation=["banking supervision", "bank supervision", "glass-steagall", "tarp", "thrift supervision", "dodd-frank",
            "financial reform", "commodity futures trading commission", "cftc", "house financial services committee",
            "basel", "capital requirement", "Volcker rule", "bank stress test", "securities and exchange commission",
            "sec", "deposit insurance", "fdic", "fslic", "ots", "occ", "firrea", "truth in lending", "union rights",
            "card check", "collective bargaining law", "national labor relations board", "nlrb", "minimum wage",
            "living wage", "right to work", "closed shop", "wages and hours", "workers compensation",
            "advance notice requirement", "affirmative action", "at-will employment", "overtime requirements",
            "trade adjustment assistance", "davis-bacon", "equal employment opportunity", "eeo", "osha", "antitrust",
            "competition policy", "merger policy", "monopoly", "patent", "copyright", "federal trade commission",
            "ftc", "unfair business practice", "cartel", "competition law", "price fixing", "class action",
            "healthcare lawsuit", "tort reform", "tort policy", "punitive damages", "medical malpractice",
            "energy policy", "energy tax", "carbon tax", "cap and trade", "cap and tax", "drilling restrictions",
            "offshore drilling", "pollution controls", "environmental restrictions", "clean air act", "clean water act",
            "environmental protection agency", "epa", "immigration policy"]

In [7]:
print(len(policy), len(regulation))

13 75


In [8]:
# Function to match whole words
def findWholeWord(w):
    return re.compile(r'\b({0})\b'.format(w), flags=re.IGNORECASE).search

In [9]:
# Function to remove multiple spaces
def remove_spaces(text):
    text=text.replace('\n', ' ').replace('\r', '')
    text=re.sub(' +',' ',text).strip()
    return text

In [10]:
# Function to lemmatize text
def lemmatize(text):
    doc=nlp(text)
    lemmas=[token.lemma_ for token in doc if not token.is_space]
    text_lemmatized=' '.join(lemmas)
    return text_lemmatized

In [11]:
# Function to find keywords in text
def find_keyword(text,keywords1=uncertain,keywords2=econ,keywords3=policy,keywords4=regulation):
    text_lemma=lemmatize(text)
    uncertain_count=0
    uncertain_words=[]
    for term in keywords1:
        term_lemma=lemmatize(term)
        if findWholeWord(term_lemma)(text_lemma)!=None:
            uncertain_count += 1
            uncertain_words.append(term)
    
    econ_count=0
    econ_words=[]
    for term in keywords2:
        term_lemma=lemmatize(term)
        if findWholeWord(term_lemma)(text_lemma)!=None:
            econ_count += 1
            econ_words.append(term)
            
    policy_count=0
    policy_words=[]
    for term in keywords3:
        term_lemma=lemmatize(term)
        if findWholeWord(term_lemma)(text_lemma)!=None:
            policy_count += 1
            policy_words.append(term)
            
    reg_count=0
    reg_words=[]
    for term in keywords4:
        term_lemma=lemmatize(term)
        if findWholeWord(term_lemma)(text_lemma)!=None:
            reg_count += 1
            reg_words.append(term)            
    
    results=[uncertain_count, uncertain_words,econ_count,econ_words,policy_count,policy_words,reg_count,reg_words]
    return results

In [ ]:
# Match keywords
uncertain_count=[]
uncertain_words=[]
econ_count=[]
econ_words=[]
policy_count=[]
policy_words=[]
reg_count=[]
reg_words=[]
for text in df['Text']:
    results=find_keyword(text)
    uncertain_count.append(results[0])
    uncertain_words.append(results[1])
    econ_count.append(results[2])
    econ_words.append(results[3])    
    policy_count.append(results[4])
    policy_words.append(results[5])    
    reg_count.append(results[6])
    reg_words.append(results[7])    

In [ ]:
df['BBDUncertainCount']=uncertain_count
df['BBDUncertainWords']=uncertain_words

In [ ]:
df['BBDEconCount']=econ_count
df['BBDEconWords']=econ_words

In [ ]:
df['BBDPolicyCount']=policy_count
df['BBDPolicyWords']=policy_words

In [ ]:
df['BBDRegCount']=reg_count
df['BBDRegWords']=reg_words

In [29]:
df.loc[(df['BBDUncertainCount']>0) & (df['BBDEconCount']>0) & (df['BBDPolicyCount']>0) & (df['BBDRegCount']>0),
      'BBD_REPU']=1
print(df['BBD_REPU'].value_counts())

1.0    4829
Name: BBD_REPU, dtype: int64


In [19]:
for i in range(0,100):
    print(df['ID'][i],uncertain_words[i],econ_words[i],policy_words[i],reg_words[i])

857780280 [] [] ['regulation', 'legislation', 'congress', 'Federal Reserve', 'regulations', 'regulatory', 'legislative'] ['house financial services committee']
925807674 [] ['economic', 'economy'] ['regulation', 'regulations'] []
430212985 ['uncertain'] ['economic', 'economy'] [] []
292620682 [] [] ['regulation', 'regulations', 'regulatory'] []
307490698 [] ['economic', 'economy'] ['regulation', 'deficit', 'congress', 'white house', 'regulations', 'deficits'] []
282723611 [] [] ['regulation', 'legislation', 'congress', 'regulations'] ['antitrust']
307420103 [] ['economic'] ['congress', 'regulatory', 'congressional'] ['deposit insurance', 'occ']
307576786 [] [] [] []
1944990599 [] ['economic', 'economy'] ['regulation', 'regulations'] []
756152643 [] [] ['regulation', 'legislation', 'regulations'] ['tarp']
837437739 [] [] ['regulation', 'regulations'] []
307276567 [] [] ['deficit', 'deficits'] []
306730233 [] ['economic'] ['regulation', 'deficit', 'congress', 'regulations', 'deficits'] [

In [21]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 788516 entries, 0 to 788515
Data columns (total 21 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   ID                 788516 non-null  object        
 1   Title              788516 non-null  object        
 2   Type               788516 non-null  object        
 3   StartDate          788516 non-null  datetime64[ns]
 4   EndDate            788516 non-null  object        
 5   Text               788516 non-null  object        
 6   TextWordCount      788516 non-null  object        
 7   PubTitle           788516 non-null  object        
 8   SourceType         788516 non-null  object        
 9   Year               788516 non-null  int64         
 10  Month              788516 non-null  int64         
 11  Newspaper          788516 non-null  object        
 12  BBDUncertainCount  788516 non-null  int64         
 13  BBDUncertainWords  788516 non-null  object  

In [20]:
df[['ID','BBDUncertainCount','BBDUncertainWords','BBDEconCount','BBDEconWords','BBDPolicyCount','BBDPolicyWords',
   'BBDRegCount','BBDRegWords']].to_csv('/home/ec2-user/SageMaker/New Uncertainty/Sentiment Analysis/BBDRegUncertaintyWords.csv',index=False)

## 3. Calculate BBD uncertainty index

In [30]:
# Calculate BBD monthly article counts by newspaper
monthlyCount=df[['Newspaper','Year','Month','BBD_REPU']].groupby(['Newspaper','Year','Month']).agg('sum').reset_index()
print(monthlyCount)

                Newspaper  Year  Month  BBD_REPU
0            Boston Globe  1985      1       0.0
1            Boston Globe  1985      2       0.0
2            Boston Globe  1985      3       1.0
3            Boston Globe  1985      4       0.0
4            Boston Globe  1985      5       0.0
...                   ...   ...    ...       ...
2940  Wall Street Journal  2020      4       4.0
2941  Wall Street Journal  2020      5       4.0
2942  Wall Street Journal  2020      6       6.0
2943  Wall Street Journal  2020      7       7.0
2944  Wall Street Journal  2020      8       7.0

[2945 rows x 4 columns]


In [31]:
# Import total monthly article counts
totalMonthlyCount=pd.read_csv('/home/ec2-user/SageMaker/New Uncertainty/Sentiment Analysis/totalMonthlyArticleCount.csv')
print(totalMonthlyCount.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4339 entries, 0 to 4338
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Newspaper            4339 non-null   object
 1   Year                 4339 non-null   int64 
 2   Month                4339 non-null   int64 
 3   Total article count  4339 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 135.7+ KB
None


In [32]:
# Merge total article count and BBD article count
totalMonthlyCount.loc[totalMonthlyCount['Newspaper']=='LA Times','Newspaper']='Los Angeles Times'
totalMonthlyCount.loc[totalMonthlyCount['Newspaper']=='Washington Post','Newspaper']='The Washington Post'
mergedMonthlyCount=monthlyCount.merge(totalMonthlyCount,on=['Newspaper','Year','Month'],how='left')
mergedMonthlyCount['year-month']=mergedMonthlyCount['Year'].map(str)+'-'+mergedMonthlyCount['Month'].map(str)

In [33]:
# Function to calculate BBD index
newspapers=mergedMonthlyCount['Newspaper'].unique()
BBDindex=mergedMonthlyCount[['Year','Month','year-month']].drop_duplicates().reset_index(drop=True)

In [34]:
T1_start="1985-1"
T1_end="2009-12"
T2_start="1985-1"
T2_end="2009-12"
def BBD_index(var):
    mergedMonthlyCount['X']=mergedMonthlyCount[var]/mergedMonthlyCount['Total article count']
    # Standardization over T1
    for newspaper in newspapers:
        mergedMonthlyCount.loc[mergedMonthlyCount['Newspaper']==newspaper,'variance']=\
            np.var(mergedMonthlyCount[(mergedMonthlyCount['Newspaper']==newspaper) &
            (T1_start <= mergedMonthlyCount['year-month']) & (mergedMonthlyCount['year-month'] <= T1_end)]['X'])
    mergedMonthlyCount['Y']=mergedMonthlyCount['X']/np.sqrt(mergedMonthlyCount['variance'])
    # Multi-paper index
    for month in BBDindex['year-month']:
        BBDindex.loc[BBDindex['year-month'] == month, 'Z'] = \
            np.mean(mergedMonthlyCount[mergedMonthlyCount['year-month'] == month]['Y'])
    # Normalization over T2
    M=np.mean(BBDindex.loc[(T2_start<=BBDindex['year-month']) & (BBDindex['year-month']<=T2_end),'Z'])
    new_var=BBDindex['Z']*(100/M)
    return new_var

In [35]:
BBDindex['BBD_REPU']=BBD_index('BBD_REPU')

In [38]:
print(BBDindex)

     Year  Month year-month         Z    BBD_REPU
0    1985      1     1985-1  0.487257   60.988212
1    1985      2     1985-2  0.122793   15.369613
2    1985      3     1985-3  1.053601  131.875514
3    1985      4     1985-4  0.646527   80.923492
4    1985      5     1985-5  0.868745  108.737733
..    ...    ...        ...       ...         ...
423  2020      4     2020-4  4.730088  592.048140
424  2020      5     2020-5  3.946027  493.910134
425  2020      6     2020-6  2.807571  351.413643
426  2020      7     2020-7  3.674267  459.894819
427  2020      8     2020-8  4.257283  532.868874

[428 rows x 5 columns]


In [37]:
BBDindex.drop('Z',axis=1).to_csv('/home/ec2-user/SageMaker/New Uncertainty/Sentiment Analysis/BBDRegUncertaintyIndex.csv',index=False)